In [ ]:
import yaml
import os

def create_data_list(source_file):
    with open(source_file, "r") as my_file:
        sources = yaml.safe_load(my_file)
    return(sources)

sources = create_data_list(source_file = "sources.yml")

In [ ]:
urls = create_data_list("sources.yml")

In [ ]:
list(urls['airports'].values())

In [ ]:
import pandas as pd

data = pd.read_csv(
    list(urls['airports'].values())[0],
    sep = ";",
    dtype = {
    "ANMOIS": "str",  
    "APT": "str",     
    "APT_NOM": "str", 
    "APT_ZON": "str",
}
)
data["an"] = data["ANMOIS"].str.slice(stop = 4)
data["mois"] = data["ANMOIS"].str.slice(start = 4)


In [ ]:
def clean_dataframe(df):

    df["an"] = df["ANMOIS"].str.slice(stop = 4)
    df["mois"] = df["ANMOIS"].str.slice(start = 4)

    # Remove leading zeros from 'mois' column
    df['mois'] = df['mois'].str.replace(r'^0+', '', regex=True)
    
    df.columns = [x.lower() for x in df.columns]

    return(df)

In [ ]:
def import_airport_data(list_files):
    # Define the data types for each column
    col_types = {
        "ANMOIS": "str",
        "APT": "str",     # equivalent to col_character()
        "APT_NOM": "str", # equivalent to col_character()
        "APT_ZON": "str", # equivalent to col_character()
    }

    # Read the CSV file(s) with the specified column types
    pax_apt_all = pd.concat([
        pd.read_csv(file, delimiter = ';', dtype = col_types)
        for file in list_files
        ])

    # Clean the DataFrame (assuming clean_dataframe is a predefined function)
    pax_apt_all = clean_dataframe(pax_apt_all)

    return pax_apt_all



def import_compagnies_data(list_files):
    # Define the data types for each column
    col_types = {
        "ANMOIS": "str",
        "CIE": "str",
        "CIE_NOM": "str",
        "CIE_NAT": "str",
        "CIE_PAYS": "str"
    }

    # Read the CSV file(s) with the specified column types
    pax_cie_all = pd.concat([
        pd.read_csv(file, delimiter = ';', dtype = col_types)
        for file in list_files
        ])

    # Clean the DataFrame (assuming clean_dataframe is a predefined function)
    pax_cie_all = clean_dataframe(pax_cie_all)


    return pax_cie_all


def import_liaisons_data(list_files):
    # Define the data types for each column
    col_types = {
        "ANMOIS": "str",
        "LSN": "str",
        "LSN_DEP_NOM": "str",
        "LSN_ARR_NOM": "str",
        "LSN_SCT": "str",
        "LSN_FSC": "str"
    }

    # Read the CSV file(s) with the specified column types
    pax_lsn_all = pd.concat([
        pd.read_csv(file, delimiter = ';', dtype = col_types)
        for file in list_files
        ])

    # Clean the DataFrame
    pax_lsn_all = clean_dataframe(pax_lsn_all)

    return pax_lsn_all


In [ ]:
pax_apt_all = import_airport_data(list(urls['airports'].values()))
pax_cie_all = import_compagnies_data(list(urls['compagnies'].values()))
pax_lsn_all = import_liaisons_data(list(urls['liaisons'].values()))


In [ ]:
import geopandas as gpd

airports_location = gpd.read_file(urls['geojson']['airport'])

In [ ]:
import folium 
m = folium.Map()

folium.GeoJson(airports_location).add_to(m)
m

In [45]:
import src.import_data as sid

In [46]:
pax_apt_all.head()

,anmois,apt,apt_nom,apt_zon,apt_peq,apt_pax_dep,apt_pax_arr,apt_pax_tr,apt_frp_dep,apt_frp_arr,apt_nmvt_mxt,apt_nmvt_cgo,an,mois
0,201801,FMCZ,MAYOTTE-MARCEL HENRY,OM,35209,13108,18977,1202,"21,8","170,4",546,1,2018,1
1,201801,FMEE,LA REUNION-ROLAND GARROS,OM,269383,112430,127167,4768,743,"1758,7",1149,16,2018,1
2,201801,FMEP,ST-PIERRE-PIERREFONDS,OM,13219,6034,7181,0,0,"0,4",208,0,2018,1
3,201801,LFAQ,ALBERT-BRAY,MT,2108,119,124,0,108,"78,5",32,15,2018,1
4,201801,LFBA,AGEN-LA GARENNE,MT,2513,1335,1178,0,0,0,131,0,2018,1


In [47]:
liste_aeroports = pax_apt_all['apt'].unique()
default_airport = liste_aeroports[0]
default_airport

'FMCZ'